<h1> IMR 10K Pen Results (2019-12-18) </h1>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from aquabyte.visualize import Visualizer
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
import json, os
import cv2
import torch
import pytz
from PIL import Image
import datetime as dt
import dateutil


In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations
    where pen_id=61
    and keypoints is not null
    and keypoints -> 'leftCrop' is not null
    and keypoints -> 'rightCrop' is not null
    and captured_at between '2019-11-01' and '2019-11-08'
    and is_qa=FALSE;
"""


df = rds_access_utils.extract_from_database(query)

In [ ]:
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()
to_tensor_transform = ToTensor()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')

In [ ]:
def generate_weight(row_id, keypoints, cm):
    
    # run biomass estimation
    input_sample = {
        'keypoints': keypoints,
        'cm': cm,
        'stereo_pair_id': row_id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    
    return weight_prediction

In [ ]:
weights = []
count = 0
for idx, row in df.iterrows():
    if count % 1000 == 0:
        print(count)
    count += 1
    weight = generate_weight(row.id, row.keypoints, row.camera_metadata)
    weights.append(weight)


In [ ]:
df['weight'] = weights

In [ ]:
plt.hist(df[(df.captured_at > '2019-11-07') & (df.captured_at < '2019-11-08')].weight, bins=100)
plt.show()

In [ ]:
def get_world_keypoints(row):
    if 'leftCrop' in row.keypoints and 'rightCrop' in row.keypoints:
        return pixel2world(row.keypoints['leftCrop'], row.keypoints['rightCrop'], row.camera_metadata)
    else:
        return None
    
df['world_keypoints'] = df.apply(
    lambda x: get_world_keypoints(x), axis=1
)

In [ ]:
wkps = df.world_keypoints.iloc[0]
v = wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']
np.arctan((v[1] / v[0])) * (180.0 / np.pi)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.theta, bins=100)
plt.grid()
plt.show()

In [ ]:
def compute_angle(wkps):
    v = wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']
    theta = np.arctan((v[1] / v[0])) * (180.0 / np.pi)
    return theta
    

df['theta'] = df.world_keypoints.apply(lambda x: compute_angle(x))

In [ ]:
df[df.theta.abs() < 15].weight.mean()

In [ ]:
v = Visualizer(s3_access_utils, rds_access_utils)

In [ ]:
mask = df.theta.abs() < 15
keypoint_annotation_id = int(df[mask].id.iloc[5005])
v.load_data(keypoint_annotation_id)
v.display_crops()

In [ ]:
v.load_data(keypoint_annotation_id)

In [ ]:
df[mask].shape

In [ ]:
df['weight'] = weights

In [ ]:
df[mask].weight.mean()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df[mask].weight, bins=10)
plt.title('Predicted weight distribution for IMR 10K pen')
plt.xlabel('Predicted weight (g)')
plt.ylabel('Frequency')
plt.grid()
plt.show()

In [ ]:
tdf = df.weight.resample('D', how=lambda x: np.mean(x))
for d in range(1, 6):
    fig, ax = plt.subplots(figsize=(10, 5))
    kdf = tdf.rolling(d).mean()
    ax.scatter(kdf.index, kdf.values)
    ax.set_title('Blom Kjeppevikholmen pen_id=5 (Pen #3): {} day aggregation'.format(d))
    ax.set_xlabel('Date')
    ax.set_ylabel('Predicted Weight (g)')
    ax.grid()
    fig.autofmt_xdate()
    plt.show()

In [ ]:
(df.adj_weight.mean() - 2070)/2070

In [ ]:
plt.figure(figsize=(20, 10))
df.adj_weight.plot.hist(bins=10)
plt.title('Biomass Results on 9/13-9/21, 9/18 excluded, manual annotations + latest biomass NN')
plt.xlabel('Predicted Weight (g)')
plt.ylabel('Frequency')
plt.grid()
plt.show()